In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html#model-layers

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [7]:
# setting up a class
#   1. subclass from nn.Module
#   2. initialize layers in __init__
#   3. implement operations on input data in the forward method

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # MNIST image size as input
            nn.ReLU(), # add the relu activation function
            nn.Linear(512, 512), # fully connected layer with the (input size, output size)
            nn.ReLU(), # add the relu activation function to the outputs
            nn.Linear(512, 10), # fully connected with (input size, output size)
            # note no activation on the end - nn.CrossEntropyLoss internally applies log_softmax
        )
    
    def forward(self, x): # note, do not call directly! instead pass to the object
        x = self.flatten(x) # take the 2d tensor image and convert to 1d tensor
        logits = self.linear_relu_stack(x)
        return logits # returns a 2d tensor - dim=0 the index of output, dim=1 the value of the output

In [8]:
# create model, move to device, and print structure
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
# Need to add softmax to get prediction probabilities for the classes from MNIST
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


In [10]:
# looking at layers using sample minibatch of 3 images of size (28,28)
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [11]:
# flatten converts 2d (28,28) to 1d (784,) - retaining the minibatch dimension
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [12]:
# linear layer is a fully connected part of NN
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
# nonlinear activations are applied after linear (fully connected) or other transformations between layers
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 3.0641e-01,  6.1469e-01, -1.1619e-01, -2.1788e-01,  5.5430e-01,
         -5.6826e-02, -6.5653e-02, -6.8821e-02, -1.9113e-01,  2.6117e-01,
          4.7677e-01,  3.7927e-04, -1.6366e-02, -8.2226e-02,  1.0688e-01,
          1.9652e-01,  3.9228e-01, -3.2533e-01, -1.3845e-01, -3.8337e-01],
        [ 2.3194e-01,  4.3532e-01,  4.1137e-02, -7.7636e-01,  2.4027e-01,
          9.9707e-02,  2.7089e-02,  8.4383e-02, -2.5856e-01,  4.8859e-02,
          4.7725e-01,  1.5629e-01,  2.9210e-01, -5.5869e-02,  1.5243e-01,
          7.1445e-01,  1.9363e-01, -4.2391e-02, -4.9109e-01, -1.8783e-01],
        [ 6.9047e-03,  4.9769e-01, -8.9560e-02, -5.4732e-01,  4.0438e-01,
         -2.0350e-02, -1.8790e-02, -2.8330e-01, -5.9206e-02,  3.8581e-02,
          5.0560e-01, -3.5624e-01, -2.5388e-02, -2.1882e-01,  2.2242e-01,
          3.9116e-01,  1.6018e-01, -2.8177e-01, -2.2700e-01,  3.4464e-03]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[3.0641e-01, 6.1469e-01, 0.0000e+00, 0.0

In [14]:
# nn.Sequential - ordered container of modules, data passed through in order defined
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [15]:
# softmax - converts logits (i.e., values in [-infty, infty] to [0,1]), dim is along which dim the values must
# sum to one
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [16]:
# layers are parameterized (with weights and biases that are optimized during training)
# - subclassing from nn.Module automatically tracks all fields into parameters()
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0336,  0.0346, -0.0307,  ...,  0.0025, -0.0115,  0.0314],
        [-0.0317,  0.0081, -0.0250,  ...,  0.0175, -0.0060,  0.0324]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0185, -0.0182], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0247,  0.0085, -0.0249,  ..., -0.0359,  0.0168, -0.0281],
        [ 0.0152,  0.0376,  0.0213,  ...,  0.0244, -0.0195, -0.0251]],
       device='cuda:0', grad_fn=<Sl